# US Accidents Dataset - Initial Loading and Exploration

**Objective**: Load the US Accidents dataset and perform initial exploration to understand the data structure, quality, and key characteristics.

**Dataset**: US Accidents (2016-2023)
- Source: Kaggle (sobhanmoosavi/us-accidents)
- Size: 2.9 GB, 7.7M+ records, 46 columns
- Geographic Coverage: 49 US states

## Notebook Contents
1. Environment Setup
2. Data Loading
3. Basic Statistics
4. Data Quality Assessment
5. Geographic Coverage
6. Temporal Patterns
7. Severity Distribution
8. Initial Findings

## 1. Environment Setup

In [ ]:
# Import required libraries
from enum import IntEnum
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import FuncFormatter
from pathlib import Path
import sys

print("✓ Required packages imported")

In [ ]:
# Add project root to path
project_root = Path().absolute().parent
sys.path.insert(0, str(project_root))

from config import Config

# Setup
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.2f}'.format)

print("✓ Environment setup complete")
print(f"Project root: {project_root}")

## 2. Data Loading

Let's load the dataset. We initially define optimized datatypes for improved storage & processing

In [ ]:
# Show file sizes
fars_csv_names = ["accident", "drugs", "person"]
class FARS_DATA_INDEX(IntEnum):
    ACCIDENT = 0
    DRUGS = 1
    PERSON = 2
years = [2018, 2019, 2023]

for year in years:
    for filename in fars_csv_names:
        data_path = Config.FARS_RAW_DIR / f"{year}/{filename}.csv"
        print(f"{year} {filename} file size: {data_path.stat().st_size / 1024**3:.2f} GB")

In [ ]:
# Define optimized data types
dtype_dict = {
    'ST_CASE': 'int32',
    'STATENAME': 'category',
    'PEDS': 'int16',
    'PERNOTMVIT': 'int16',
    'VE_TOTAL': 'int16',
    'VE_FORMS': 'int16',
    'PVH_INVL': 'int16',
    'PERSONS': 'int16',
    'PERMVIT': 'int16',
    'COUNTYNAME': 'string',
    'MONTH': 'int8',
    'DAY': 'int8',
    'DAY_WEEKNAME': 'category',
    'YEAR': 'int16',
    'HOUR': 'int8',
    'MINUTE': 'int8',
    'TWAY_ID': 'string',
    'ROUTENAME': 'string',
    'RUR_URBNAME': 'category',
    'FUNC_SYSNAME': 'string',#'category',
    'RD_OWNERNAME': 'category',
    'NHSNAME': 'category',
    'SP_JURNAME': 'category',
    'HARM_EVNAME': 'category',
    'MAN_COLLNAME': 'string',#'category',
    'RELJCT2NAME': 'category',
    'TYP_INTNAME': 'category',
    'REL_ROADNAME': 'category',
    'WRK_ZONE': 'category',
    'LGT_CONDNAME': 'category',
    'WEATHERNAME': 'category',
    'SCH_BUS': 'boolean',
    'RAIL': 'string',
    'NOT_HOUR': 'int8',
    'NOT_MIN': 'int8',
    'ARR_HOUR': 'int8',
    'ARR_MIN': 'int8',
    'HOSP_HR': 'int8',
    'HOSP_MN': 'int8',
    'DRUGRESNAME': 'string',#'category',
    'AGE': 'int8',
    'PER_TYPNAME': 'category',
    'INJ_SEV': 'int8',
    'ALC_RES': 'int16',
}

print("Data type optimization configured")
print(f"Categorical columns: {sum(1 for v in dtype_dict.values() if v == 'category')}")
print(f"Boolean columns: {sum(1 for v in dtype_dict.values() if v == 'bool')}")
print(f"Float32 columns: {sum(1 for v in dtype_dict.values() if v == 'float32')}")

# Load the raw data
fars_data = [[pd.DataFrame() for _ in range(len(fars_csv_names))] for _ in range(len(years))]
for i, year in enumerate(years):
    for j, filename in enumerate(fars_csv_names):
        data_path = Config.FARS_RAW_DIR / f"{year}/{filename}.csv"
        fars_data[i][j] = pd.read_csv(data_path, dtype=dtype_dict, sep=',', encoding='latin_1')
        print(f"\n✓ Dataset loaded: {data_path} {len(fars_data[i][j]):,} rows × {len(fars_data[i][j].columns)} columns")

In [ ]:
# Keep only columns of interest/drop unimportant columns
# This will probably be replaced by something determined by PCA
fars_accident_cols = [
    'ST_CASE', 'STATENAME', 'PEDS', 'PERNOTMVIT', 'VE_TOTAL', 'VE_FORMS', 'PVH_INVL', 'PERSONS', 'PERMVIT',
    'COUNTYNAME', 'MONTH', 'DAY', 'DAY_WEEKNAME', 'YEAR', 'HOUR', 'MINUTE', 'TWAY_ID', 'ROUTENAME',
    'RUR_URBNAME', 'FUNC_SYSNAME', 'RD_OWNERNAME', 'NHSNAME', 'SP_JURNAME', 'HARM_EVNAME',
    'MAN_COLLNAME', 'RELJCT2NAME', 'TYP_INTNAME', 'REL_ROADNAME', 'WRK_ZONE', 'LGT_CONDNAME', 'WEATHERNAME',
    'SCH_BUSNAME', 'RAIL', 'NOT_HOUR', 'NOT_MIN', 'ARR_HOUR', 'ARR_MIN', 'HOSP_HR', 'HOSP_MN'
    ]
fars_drugs_cols = ['ST_CASE', 'PER_NO', 'DRUGRESNAME']
fars_person_cols = ['ST_CASE', 'PER_NO', 'AGE', 'PER_TYP', 'INJ_SEV', 'ALC_RES']
fars_col_subsets = [fars_accident_cols, fars_drugs_cols, fars_person_cols]
for i, year in enumerate(years):
    for j, fars_col_subset in enumerate(fars_col_subsets):
        print(f"Dropping unwanted columns from {fars_csv_names[j]}:")
        fars_data[i][j] = fars_data[i][j][fars_col_subset]
        print(f"\n✓ Columns dropped, new dataset shape is: {len(fars_data[i][j]):,} rows × {len(fars_data[i][j].columns)} columns")

## 3. Joining FARS datasets

In [ ]:
# Missing values analysis
# TODO personrf has indications of whether or not a drug/alcohol test was refused
missing = pd.DataFrame({
        'Column': [],
        'Missing_Count': [],
        'Missing_Percentage': []
    })

remap_vals = {
    'ROUTENAME': {
        'County': 'County Road',
        'Township': 'Local Street - Township',
        'Municipal': 'Local Street - Municipality',
        'Local Street - Frontage Road': 'Other',
        'Parkway Marker or Forest Route Marker [Specify:]': 'Other',
        'Off-Interstate Business Marker': 'Other',
        'Secondary Route': 'Other',
        'Bureau of Indian Affairs': 'Other'
        },
    'FUNC_SYSNAME': {
        'Other Freeways and Expressways': 'Principal Arterial - Other Freeways and Expressways',
        'Other Principal Arterial': 'Principal Arterial - Other'
        },
    'MAN_COLLNAME': {
        'First Harmful Event was Not a Collision with Motor Vehicle In-Transport': 'Not a Collision with Motor Vehicle In-Transport'
        },
    'DRUGRESNAME': {
        'Narcotic Analgesics': 'Narcotics',
        'Phencyclidine (PCP)': 'Other Drug (Specify:)',
        'Dissociative Anesthetics': 'Other Drug (Specify:)',
        'Non-Psychoactive/Other Drugs': 'None Detected/Below Threshold',
    }
}

missing_vals = {
    'HOUR': [99],
    'MINUTE': [99],
    'TWAY_ID': [999999999999999999999999999999],
    'ROUTENAME': ['Unknown/Not Reported', 'Trafficway Not in State Inventory', 'Unknown'], #Changes 2023
    'RUR_URBNAME': [6, 8, 9],
    'FUNC_SYSNAME': [96, 98, 99], #Changes 2023
    'RD_OWNERNAME': [96, 98, 99],
    'NHSNAME': [9],
    'SP_JURNAME': [9],
    'HARM_EVNAME': ['Reported as Unknown'],
    'MAN_COLLNAME': ['Unknown'], #Changes 2019
    'RELJCT2NAME': ['Reported as Unknown', 'Not Reported'],
    'TYP_INTNAME': ['Reported as Unknown', 'Not Reported'], #Change 2020
    'REL_ROADNAME': ['Reported as Unknown', 'Not Reported'],
    'LGT_CONDNAME': ['Other', 'Not Reported', 'Reported as Unknown'],
    'WEATHERNAME': ['Not Reported', 'Unknown', 'Reported as Unknown'],
    'RAIL': ['9999999'],
    'NOT_HOUR': [88, 99],
    'NOT_MIN': [88, 98, 99],
    'DRUGRESNAME': ['Reported as Unknown if Tested for Drugs', 'Tested for Drugs, Results Unknown', 'Not Reported'], #Changes 2022
    'ALC_RES': [999], #Needs more than just dealing with missing values
    'INJ_SEV': [9],
    'AGE': [99] #97 or older are all recorded as 97
}

for i, year in enumerate(years):
    for j, dataset in enumerate(fars_data[i]):
        for column in dataset:
            if column in remap_vals:
                for value in remap_vals[column]:
                    dataset.loc[dataset[column] == value, column] = remap_vals[column][value]

            if column in missing_vals:
                for value in missing_vals[column]:
                    dataset.loc[dataset[column] == value, column] = None

print("Remapped non-missing values & replaced missing values")

for i, dataset in enumerate(fars_data[0]):
    for column in dataset:
        missing_count = sum([fars_data[j][i][column].isna().sum() for j in range(len(fars_data))])
        total_count = sum([len(fars_data[j][i][column]) for j in range(len(fars_data))])
        new_missing_frame = pd.DataFrame({
            'Column': [f"{fars_csv_names[i]}.{column}"],
            'Missing_Count': [missing_count],
            'Missing_Percentage': [((missing_count / total_count) * 100).round(2)]
        })
        missing = pd.concat([missing, new_missing_frame])

missing = missing[missing['Missing_Count'] > 0].sort_values('Missing_Percentage', ascending=False)
print(f"Columns with missing values: {len(missing)} out of {sum([len(dataset.columns) for dataset in fars_data[0]])}")
print("\nColumns with missing data:")
missing = missing.reset_index(drop=True)
display(missing)

In [ ]:
# Visualize missing data
fig, ax = plt.subplots(figsize=(12, 8))
missing_top = missing.head(10)
ax.barh(missing_top['Column'], missing_top['Missing_Percentage'], color='coral')
ax.set_xlabel('Missing Percentage (%)', fontsize=12)
ax.set_title('Top 10 Columns with Missing Data', fontsize=14, fontweight='bold')
ax.invert_yaxis()
plt.tight_layout()
plt.show()

In [ ]:
new_cols = ['TOTAL_HARM', 'INTOXICATED_DRIVER_INVOLVED']
columns = fars_accident_cols + new_cols

joined_years_data = []

for i, datasets in enumerate(fars_data):
    joined_year_data = datasets[FARS_DATA_INDEX.ACCIDENT].copy()
    for col in new_cols:
        joined_year_data[col] = pd.NA

    for j, accident in datasets[FARS_DATA_INDEX.ACCIDENT].iterrows():
        num_persons = accident['PERSONS']
        person_row_number = datasets[FARS_DATA_INDEX.PERSON].index[datasets[FARS_DATA_INDEX.PERSON]['ST_CASE'] == accident['ST_CASE']][0]
        drugs_row_number = datasets[FARS_DATA_INDEX.DRUGS]['ST_CASE'].index[datasets[FARS_DATA_INDEX.DRUGS]['ST_CASE'] == accident['ST_CASE']][0]
        driver_person_numbers = []
        intoxicated_driver_involved = False
        for column in new_cols:
            if column == new_cols[0]: #Add a sum for all injuries
                total_harm = 0
                for k in range(num_persons):
                    person_harm = datasets[FARS_DATA_INDEX.PERSON].loc[person_row_number + k, 'INJ_SEV']
                    if person_harm <= 4:
                        total_harm += person_harm
                    elif person_harm == 5: #Injured, but unknown severity
                        total_harm += 1
                joined_year_data.loc[j, column] = total_harm

            elif column == new_cols[1]: #Check if any intoxicated drivers were involved
                drunk_drivers = 0
                for k in range(num_persons):
                    bac = datasets[FARS_DATA_INDEX.PERSON].loc[person_row_number + k, 'ALC_RES']
                    per_type = datasets[FARS_DATA_INDEX.PERSON].loc[person_row_number + k, 'PER_TYP']
                    driver_person_number = datasets[FARS_DATA_INDEX.PERSON].loc[person_row_number + k, 'PER_NO']
                    if bac > 80 and bac not in list(range(995, 999)) and per_type == 1: #0.080% is the legal limit in all states, some states have additional punishments for certain higher values. Special values are assumed to be none
                        intoxicated_driver_involved = True
                        break

                    drugs_idx = drugs_row_number
                    drugs_row = datasets[FARS_DATA_INDEX.DRUGS].iloc[drugs_idx]
                    while drugs_row['ST_CASE'] == accident['ST_CASE'] and drugs_row['PER_NO'] == driver_person_number:
                        result = drugs_row['DRUGRESNAME']
                        if pd.notna(result) and (not (result in [None, 'Test Not Given', 'None Detected/Below Threshold'])):
                            intoxicated_driver_involved = True
                            break
                        if drugs_idx == len(datasets[FARS_DATA_INDEX.DRUGS]) - 1:
                            break
                        drugs_idx += 1
                        drugs_row = datasets[FARS_DATA_INDEX.DRUGS].iloc[drugs_idx]
                    
                    if intoxicated_driver_involved:
                        break

            joined_year_data.loc[j, 'INTOXICATED_DRIVER_INVOLVED'] = intoxicated_driver_involved
    joined_years_data += [joined_year_data]
joined_data = pd.concat(joined_years_data)

display(joined_data.head())

## 4. Geographic Coverage

In [ ]:
# State distribution
state_counts = joined_data['STATENAME'].value_counts().head(15)
print("Top 15 States by fatal accident Count:")
print("=" * 40)
for state, count in state_counts.items():
    pct = (count / len(joined_data)) * 100
    print(f"{state:10s} {count:>10,}  ({pct:5.2f}%)")

In [ ]:
# Visualize state distribution
fig, ax = plt.subplots(figsize=(14, 6))
state_counts.plot(kind='bar', ax=ax, color='steelblue')
ax.set_title('Top 15 States by Number of Fatal Accidents', fontsize=14, fontweight='bold')
ax.set_xlabel('State', fontsize=12)
ax.set_ylabel('Number of Fatal Accidents', fontsize=12)
ax.yaxis.set_major_formatter(FuncFormatter(lambda x, p: f'{float(x/1000)}K'))
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 5. Temporal Patterns

In [ ]:
# Accidents by hour of day
hour_counts = joined_data['HOUR'].value_counts().sort_index()

fig, ax = plt.subplots(figsize=(14, 6))
ax.plot(hour_counts.index, hour_counts.values, marker='o', linewidth=2, markersize=8, color='darkblue')
ax.fill_between(hour_counts.index, hour_counts.values, alpha=0.3)
ax.set_title('Fatal Accident Distribution by Hour of Day', fontsize=14, fontweight='bold')
ax.set_xlabel('Hour of Day', fontsize=12)
ax.set_ylabel('Number of Accidents', fontsize=12)
ax.grid(True, alpha=0.3)
ax.set_xticks(range(0, 24))
plt.tight_layout()
plt.show()

In [ ]:
# Accidents by day of week
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
day_counts = joined_data['DAY_WEEKNAME'].value_counts().reindex(day_order)

fig, ax = plt.subplots(figsize=(12, 6))
day_counts.plot(kind='bar', ax=ax, color='coral')
ax.set_title('Fatal Accident Distribution by Day of Week', fontsize=14, fontweight='bold')
ax.set_xlabel('Day of Week', fontsize=12)
ax.set_ylabel('Number of Fatal Accidents', fontsize=12)
ax.yaxis.set_major_formatter(FuncFormatter(lambda x, p: f'{int(x/1000)}K'))
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

**Observation:** The weekend has a higher number of fatal accidents compared to working days, let's dive into the hourly distribution on weekends to see why

In [ ]:
# Accidents by hour of day on weekends

hour_counts = joined_data[joined_data['DAY_WEEKNAME'].isin(['Saturday', 'Sunday'])]['HOUR'].value_counts().sort_index()

fig, ax = plt.subplots(figsize=(14, 6))
ax.plot(hour_counts.index, hour_counts.values, marker='o', linewidth=2, markersize=8, color='darkblue')
ax.fill_between(hour_counts.index, hour_counts.values, alpha=0.3)
ax.set_title('Accident Distribution by Hour of Day (Weekends)', fontsize=14, fontweight='bold')
ax.set_xlabel('Hour of Day', fontsize=12)
ax.set_ylabel('Number of Accidents', fontsize=12)
ax.grid(True, alpha=0.3)
ax.set_xticks(range(0, 24))
plt.tight_layout()
plt.show()

# Identify the peak hours for accidents

min_accidents = hour_counts.min()
peak_hours = hour_counts.sort_values(ascending=False).head(10)

print("\nTop 10 Peak Hours for Accidents:\n")
for hour, count in peak_hours.items():
    print(f"At {int(hour):02d}h: {count:,} accidents ({count/min_accidents:.2f}x minimum)")  

print(f"\nMinimum Accidents in a Single Hour (weekends): {min_accidents:,} accidents")

**Observation:** The rush hour patterns are less pronounced on weekends compared to weekdays. While there are still peaks in accident counts during late morning and early afternoon hours, the overall distribution is more uniform throughout the day. This suggests that weekend traffic is less influenced by traditional work commute times, leading to a more even spread of accidents across different hours.

## 6. Severity Distribution

In [ ]:
# Severity distribution
severity_counts = joined_data['TOTAL_HARM'].value_counts().sort_index()

print("Accident Severity Distribution:")
print("=" * 50)
print("Level | Count      | Percentage")
print("-" * 50)
for sev, count in severity_counts.items():
    pct = (count / len(joined_data)) * 100
    print(f"  {sev}   | {count:>10,} | {pct:>6.2f}%")

print("\nNote: Severity levels 4+, sum of all casualties in each fatal accident (1 = Possible injury, 2 = Suspected minor injury, 3 = Suspected serious injury, 4 = fatality)")

In [ ]:
# Visualize severity distribution
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Box plot
ax1.boxplot(joined_data['TOTAL_HARM'])
ax1.set_title('Accidents by Severity Level', fontsize=14, fontweight='bold')
ax1.semilogy()
ax1.set_xlabel('Severity', fontsize=12)
ax1.set_ylabel('Number of Accidents', fontsize=12)

# Violin plot
ax2.violinplot(joined_data['TOTAL_HARM'].astype(float), showmeans=False, showmedians=True)
ax2.set_title('Severity Distribution', fontsize=14, fontweight='bold')
ax2.semilogy()

plt.tight_layout()
plt.show()

## 7. Weather Conditions

In [ ]:
# Top weather conditions
weather_counts = joined_data['WEATHERNAME'].value_counts().head(15)

print("Top 15 Weather Conditions:")
print("=" * 60)
for i, (weather, count) in enumerate(weather_counts.items(), 1):
    pct = (count / len(joined_data)) * 100
    print(f"{i:2d}. {weather:35s} {count:>8,}  ({pct:5.2f}%)")

## 8. Infrastructure Features
Probably going to drop this, it's kind of difficult to do manually and we can probably find more useful information with some model

In [ ]:
# # Infrastructure boolean columns
# infrastructure_cols = ['Amenity', 'Bump', 'Crossing', 'Give_Way', 'Junction', 
#                        'No_Exit', 'Railway', 'Roundabout', 'Station', 'Stop', 
#                        'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop']

# # Handle missing values first by filling with False (assuming missing means feature not present)
# df_clean = df[infrastructure_cols].fillna(False)

# infra_data = []
# for col in infrastructure_cols:
#     true_count = df_clean[col].sum()
#     false_count = len(df_clean) - true_count
#     missing_count = df[col].isnull().sum()
#     total_count = len(df)
    
#     true_percentage = (true_count / total_count) * 100
#     false_percentage = (false_count / total_count) * 100
#     missing_percentage = (missing_count / total_count) * 100
    
#     infra_data.append({
#         'Feature': col,
#         'True_Count': true_count,
#         'True_Percentage': true_percentage,
#         'False_Count': false_count,
#         'False_Percentage': false_percentage,
#         'Missing_Percentage': missing_percentage
#     })

# infra_stats = pd.DataFrame(infra_data).sort_values('True_Percentage', ascending=False)

# print("Infrastructure Feature Presence:")
# print("=" * 60)
# print(infra_stats.to_string(index=False))

In [ ]:
# # Visualize infrastructure features
# fig, ax = plt.subplots(figsize=(12, 8))
# ax.barh(infra_stats['Feature'], infra_stats['True_Percentage'], color='steelblue')
# ax.set_xlabel('Percentage of Accidents (%)', fontsize=12)
# ax.set_title('Presence of Infrastructure Features in Accidents', fontsize=14, fontweight='bold')
# ax.invert_yaxis()
# plt.tight_layout()
# plt.show()

## 9. Key Findings Summary

In [ ]:
print("="*80)
print("KEY FINDINGS - FARS DATASET (2023)")
print("="*80)

print("\n📊 DATASET SIZE:")
print(f"   • Total records: {len(joined_data)}")
print(f"   • Features: {len(joined_data.columns)}")
print(f"   • Memory usage: {joined_data.memory_usage(deep=True).sum() / 1024**3:.2f} GB")

print("\n📍 GEOGRAPHIC COVERAGE:")
print(f"   • States: {joined_data['STATENAME'].nunique()}")
print(f"   • Top state: {joined_data['STATENAME'].value_counts().index[0]} ({joined_data['STATENAME'].value_counts().iloc[0]:,} accidents)")

# print("\n⚠️ SEVERITY DISTRIBUTION:")
# for sev in sorted(joined_data['TOTAL_HARM'].unique()):
#     count = (joined_data['TOTAL_HARM'] == sev).sum()
#     pct = (count / len(joined_data)) * 100
#     print(f"   • Level {sev}: {count:>10,} ({pct:>5.2f}%)")

print("\n🌤️ WEATHER CONDITIONS:")
print(f"   • Most common: {joined_data['WEATHERNAME'].value_counts().index[0]}")

# print("\n🚦 INFRASTRUCTURE:")
# print(f"   • Traffic signals: {(df['Traffic_Signal'].sum() / len(df) * 100):.1f}%")
# print(f"   • Crossings: {(df['Crossing'].sum() / len(df) * 100):.1f}%")
# print(f"   • Junctions: {(df['Junction'].sum() / len(df) * 100):.1f}%")

print("\n⚠️ DATA QUALITY:")
print(f"   • Columns with missing data: {len(missing)} / {len(joined_data.columns)}")
print(f"   • Overall completeness: {(1 - joined_data.isnull().sum().sum() / (len(joined_data) * len(joined_data.columns))) * 100:.2f}%")

print("\n" + "="*80)

## 12. Save preprocessed Dataset with Modified Data Types and New Features

Lets save the dataset with modified data types and new features for quick testing in subsequent notebooks

In [ ]:
output_full_path = Config.FARS_CLEANED_DIR / "fars.csv"
joined_data.to_csv(output_full_path, index=False)

print("\nDataset saved with the following details:")
print(f"✓ Cleaned dataset saved to: {output_full_path}")